In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df_train = pd.read_csv('training_data_final.csv')
df_val = pd.read_csv('validation_data_final.csv')

In [5]:
df_train.describe()

,Unnamed: 0,song_id,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,label,avg,first_der,Spectral_Bandwidth
count,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,...,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,2.571688e+06,2.571686e+06,2.571688e+06
mean,2.620337e+05,7.732505e+00,-1.840672e+02,1.603907e+02,-4.029035e+01,2.812147e+01,-6.071339e-01,5.265247e+00,-4.475622e+00,1.530516e+00,...,-7.745443e-01,-3.745030e+00,-5.219728e-03,-2.782827e+00,-1.662403e-01,-1.902725e+00,3.318604e+00,-3.146505e+00,-6.970197e-05,-6.632178e+00
std,1.602806e+05,6.753933e+00,9.446983e+01,4.454711e+01,3.859133e+01,3.430488e+01,2.370341e+01,2.064640e+01,1.627559e+01,1.530715e+01,...,1.039811e+01,9.794225e+00,1.080832e+01,9.979153e+00,1.014578e+01,1.014417e+01,1.787124e+00,6.218659e+00,1.194906e+00,2.564719e+01
min,0.000000e+00,1.000000e+00,-6.165092e+02,-1.189609e+02,-1.615610e+02,-1.101264e+02,-9.880458e+01,-7.487053e+01,-8.693895e+01,-7.969452e+01,...,-5.290346e+01,-5.996405e+01,-5.287412e+01,-5.433993e+01,-4.833425e+01,-5.187897e+01,0.000000e+00,-3.082546e+01,-1.931720e+01,-1.541273e+02
25%,1.229280e+05,1.000000e+00,-2.449527e+02,1.333805e+02,-6.828790e+01,5.249542e+00,-1.753157e+01,-9.742997e+00,-1.515791e+01,-8.800862e+00,...,-7.642842e+00,-1.021720e+01,-7.213601e+00,-9.262317e+00,-6.885947e+00,-8.560415e+00,2.000000e+00,-7.001982e+00,-6.254794e-01,-1.760693e+01
50%,2.568690e+05,6.000000e+00,-1.725987e+02,1.645124e+02,-4.266308e+01,3.391434e+01,-9.043165e-02,4.512479e+00,-4.130829e+00,1.065307e+00,...,-9.121526e-01,-4.229023e+00,-5.039742e-01,-3.357756e+00,-6.998017e-01,-2.673890e+00,3.000000e+00,-2.625020e+00,4.624171e-02,-3.497698e+00
75%,3.911380e+05,1.400000e+01,-1.156562e+02,1.928420e+02,-1.445634e+01,5.286643e+01,1.618911e+01,2.004590e+01,6.461272e+00,1.163819e+01,...,5.605886e+00,1.947588e+00,6.439716e+00,2.801736e+00,5.790803e+00,3.614494e+00,5.000000e+00,1.157240e+00,7.127167e-01,1.009997e+01
max,6.164320e+05,2.300000e+01,9.327194e+01,2.762949e+02,1.164280e+02,1.544931e+02,9.654688e+01,1.008150e+02,7.274289e+01,7.844859e+01,...,7.515225e+01,6.366040e+01,6.902464e+01,6.572739e+01,6.863934e+01,6.387963e+01,6.000000e+00,1.427362e+01,2.516984e+01,5.784696e+01


In [ ]:
import numpy as np
import pandas as pd
import librosa

# Assuming df has columns named 0 to 19 for the 20 MFCC coefficients and other columns like label
mfcc_columns = [str(i) for i in range(20)]  # Column names for the MFCC coefficients
mfcc_data = np.array(df_train[mfcc_columns])  # Convert the MFCC columns to a NumPy array

# 1. Autocorrelation Features (calculated per row)
def autocorrelation_features(mfcc_data):
    peak_values, peak_lags, sum_autocorrs = [], [], []
    
    for row in mfcc_data:
        peak_vals_row, peak_lags_row, sum_autocorrs_row = [], [], []
        for coeff in row:
            autocorr = librosa.autocorrelate(np.array([coeff]))

            if len(autocorr) > 1:
                peak_value = np.max(autocorr[1:])  # Exclude lag 0
                peak_lag = np.argmax(autocorr[1:]) + 1
                sum_autocorr = np.sum(autocorr)
            else:
                peak_value, peak_lag, sum_autocorr = 0, 0, 0
            
            peak_vals_row.append(peak_value)
            peak_lags_row.append(peak_lag)
            sum_autocorrs_row.append(sum_autocorr)
        
        peak_values.append(np.mean(peak_vals_row))
        peak_lags.append(np.mean(peak_lags_row))
        sum_autocorrs.append(np.mean(sum_autocorrs_row))
    
    return peak_values, peak_lags, sum_autocorrs

# 2. Spectral Centroid and Bandwidth
freq_centers = np.linspace(200, 8000, 20)  # Example center frequencies for 20 MFCC coefficients

def spectral_features(mfcc_data, freq_centers):
    centroids, bandwidths = [], []
    
    for mfcc_values in mfcc_data:
        if np.sum(mfcc_values) == 0:
            centroids.append(0)
            bandwidths.append(0)
            continue

        centroid = np.sum(freq_centers * mfcc_values) / np.sum(mfcc_values)
        centroids.append(centroid)
        
        bandwidth = np.sqrt(np.sum(((freq_centers - centroid) ** 2) * mfcc_values) / np.sum(mfcc_values))
        bandwidths.append(bandwidth)
    
    return centroids, bandwidths

# 3. Energy and Power Calculation (per row)
def compute_energy_and_power(mfcc_data):
    energies = np.sum(mfcc_data ** 2, axis=1)  # Sum of squares per observation
    powers = np.mean(mfcc_data ** 2, axis=1)   # Mean of squares per observation
    return energies, powers

# 4. Zero-Crossing Rate Calculation (per row) with smaller frame_length
def zero_crossing_rate(mfcc_data, frame_length=20):
    zcrs = []
    for row in mfcc_data:
        zcr_row = np.mean(librosa.feature.zero_crossing_rate(row.reshape(1, -1), frame_length=frame_length, hop_length=frame_length // 2, center=False))
        zcrs.append(zcr_row)
    return zcrs

# Applying feature engineering
peak_values, peak_lags, sum_autocorrs = autocorrelation_features(mfcc_data)
centroids, bandwidths = spectral_features(mfcc_data, freq_centers)
energies, powers = compute_energy_and_power(mfcc_data)
zcrs = zero_crossing_rate(mfcc_data)

# Adding engineered features to DataFrame
df_train['autocorr_peak_values'] = peak_values
df_train['autocorr_peak_lags'] = peak_lags
df_train['autocorr_sum'] = sum_autocorrs
df_train['spectral_centroid'] = centroids
df_train['spectral_bandwidth'] = bandwidths
df_train['energy'] = energies
df_train['power'] = powers
df_train['zero_crossing_rate'] = zcrs

# The final DataFrame df will now include original columns, labels, and the new feature-engineered columns.

In [ ]:
import numpy as np
import pandas as pd
import librosa

# Assuming df has columns named 0 to 19 for the 20 MFCC coefficients and other columns like label
mfcc_columns = [str(i) for i in range(20)]  # Column names for the MFCC coefficients
mfcc_data = np.array(df_val[mfcc_columns])  # Convert the MFCC columns to a NumPy array

# 1. Autocorrelation Features (calculated per row)
def autocorrelation_features(mfcc_data):
    peak_values, peak_lags, sum_autocorrs = [], [], []
    
    for row in mfcc_data:
        peak_vals_row, peak_lags_row, sum_autocorrs_row = [], [], []
        for coeff in row:
            autocorr = librosa.autocorrelate(np.array([coeff]))

            if len(autocorr) > 1:
                peak_value = np.max(autocorr[1:])  # Exclude lag 0
                peak_lag = np.argmax(autocorr[1:]) + 1
                sum_autocorr = np.sum(autocorr)
            else:
                peak_value, peak_lag, sum_autocorr = 0, 0, 0
            
            peak_vals_row.append(peak_value)
            peak_lags_row.append(peak_lag)
            sum_autocorrs_row.append(sum_autocorr)
        
        peak_values.append(np.mean(peak_vals_row))
        peak_lags.append(np.mean(peak_lags_row))
        sum_autocorrs.append(np.mean(sum_autocorrs_row))
    
    return peak_values, peak_lags, sum_autocorrs

# 2. Spectral Centroid and Bandwidth
freq_centers = np.linspace(200, 8000, 20)  # Example center frequencies for 20 MFCC coefficients

def spectral_features(mfcc_data, freq_centers):
    centroids, bandwidths = [], []
    
    for mfcc_values in mfcc_data:
        if np.sum(mfcc_values) == 0:
            centroids.append(0)
            bandwidths.append(0)
            continue

        centroid = np.sum(freq_centers * mfcc_values) / np.sum(mfcc_values)
        centroids.append(centroid)
        
        bandwidth = np.sqrt(np.sum(((freq_centers - centroid) ** 2) * mfcc_values) / np.sum(mfcc_values))
        bandwidths.append(bandwidth)
    
    return centroids, bandwidths

# 3. Energy and Power Calculation (per row)
def compute_energy_and_power(mfcc_data):
    energies = np.sum(mfcc_data ** 2, axis=1)  # Sum of squares per observation
    powers = np.mean(mfcc_data ** 2, axis=1)   # Mean of squares per observation
    return energies, powers

# 4. Zero-Crossing Rate Calculation (per row) with smaller frame_length
def zero_crossing_rate(mfcc_data, frame_length=20):
    zcrs = []
    for row in mfcc_data:
        zcr_row = np.mean(librosa.feature.zero_crossing_rate(row.reshape(1, -1), frame_length=frame_length, hop_length=frame_length // 2, center=False))
        zcrs.append(zcr_row)
    return zcrs

# Applying feature engineering
peak_values, peak_lags, sum_autocorrs = autocorrelation_features(mfcc_data)
centroids, bandwidths = spectral_features(mfcc_data, freq_centers)
energies, powers = compute_energy_and_power(mfcc_data)
zcrs = zero_crossing_rate(mfcc_data)

# Adding engineered features to DataFrame
df_val['autocorr_peak_values'] = peak_values
df_val['autocorr_peak_lags'] = peak_lags
df_val['autocorr_sum'] = sum_autocorrs
df_val['spectral_centroid'] = centroids
df_val['spectral_bandwidth'] = bandwidths
df_val['energy'] = energies
df_val['power'] = powers
df_val['zero_crossing_rate'] = zcrs

# The final DataFrame df will now include original columns, labels, and the new feature-engineered columns.

In [ ]:
df_train_new = df_train

In [ ]:
df_train_new.drop('autocorr_peak_values', axis=1)
df_train_new.drop('autocorr_peak_lags', axis=1)
df_train_new.drop('autocorr_sum', axis=1)
df_train_new.drop('avg', axis=1)
df_train_new.drop('first_der', axis=1)

In [ ]:
df